In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

%pip install mlxtend

In [5]:
df = pd.read_csv("Life Expectancy Data.csv")

In [6]:
y = df["Life expectancy "]
X =  df.drop(["Life expectancy "], axis=1)

In [7]:
y.fillna(y.median(), inplace=True)

In [8]:
X.Year = pd.to_datetime(X.Year).dt.year

In [9]:
bin_enc = ce.BinaryEncoder(drop_invariant=True)
X = bin_enc.fit_transform(X) 

In [10]:
X.fillna(X.mean(), inplace=True)

In [11]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.30, random_state=9)

## Случайный лес (группа бэггинга)

Создается несколько независимых моделей на основе одного и того же алгоритма, но на разных наборах данных, с использованием случайной подвыборки из общего набора данных. Затем предсказания всех этих моделей комбинируются в одно общее предсказание путем усреднения или голосования.

Случайный лес - это метод ансамблирования в машинном обучении, который состоит из нескольких деревьев решений. Каждое дерево решений обучается на подмножестве данных и подмножестве признаков, которые выбираются случайным образом. Когда нужно сделать прогноз, каждое дерево в лесу выдает свой отдельный прогноз, а затем все прогнозы объединяются в один общий прогноз путем усреднения или голосования.

In [9]:
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
prediction = rfr.predict(X_test)

In [11]:
params = { 
 'max_depth' : [10, 15, 20],
}

grid = GridSearchCV(estimator=RandomForestRegressor(), param_grid=params, cv= 5)
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 15, 20]})

In [12]:
grid.best_score_, grid.best_params_

(0.9562692783042547, {'max_depth': 15})

In [13]:
rfr = RandomForestRegressor(max_depth=15, random_state=0)
rfr.fit(X_train, y_train)
prediction = rfr.predict(X_test)
mean_squared_error(y_test, prediction)

5.7690366227770795

## Группа бустинга

Последовательное обучение слабых моделей для создания сильной модели. В бустинге каждая модель учится на наборе данных, который был скорректирован на основе ошибок предыдущей модели. Таким образом, каждая следующая модель фокусируется на тех объектах, на которых предыдущая модель ошиблась, и старается исправить ошибки. В результате мы получаем сильную модель, которая может давать более точные прогнозы, чем каждая из слабых моделей в отдельности.

In [14]:
params = {
    'n_estimators': [500, 800],
    'max_depth': [5, 8],
    'min_samples_split': [2, 5],
    'learning_rate': [0.01, 0.1]
}
grid = GridSearchCV(GradientBoostingRegressor(), param_grid=params, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1], 'max_depth': [5, 8],
                         'min_samples_split': [2, 5],
                         'n_estimators': [500, 800]})

In [15]:
grid.best_score_, grid.best_params_

(0.9574101819404645,
 {'learning_rate': 0.1,
  'max_depth': 5,
  'min_samples_split': 5,
  'n_estimators': 800})

In [16]:
gbr = GradientBoostingRegressor(**grid.best_params_)
gbr.fit(X_train, y_train)
prediction = gbr.predict(X_test)

In [17]:
mean_squared_error(y_test, prediction)

3.4616492010705873

## Группа стекинга

Обучаем несколько моделей на тренировочном наборе данных, затем используем их для создания прогнозов на тестовом наборе данных. Затем мы собираем прогнозы от базовых моделей и используем их в качестве входных данных для мета-модели. Мета-модель обучается на этих данных, чтобы улучшить производительность модели.

In [16]:
reg1 = RandomForestRegressor(random_state=42)
reg2 = GradientBoostingRegressor(random_state=42)
reg3 = LinearRegression()
meta_learner = LinearRegression()
sr = StackingCVRegressor(regressors=[reg1, reg2, reg3], meta_regressor=meta_learner)
params = {
    'randomforestregressor__max_depth': [5, 8]
}

grid = GridSearchCV(estimator=sr, param_grid=params, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=StackingCVRegressor(meta_regressor=LinearRegression(),
                                           regressors=[RandomForestRegressor(random_state=42),
                                                       GradientBoostingRegressor(random_state=42),
                                                       LinearRegression()]),
             n_jobs=-1,
             param_grid={'randomforestregressor__max_depth': [5, 8]})

In [20]:
sr.get_params().keys()

dict_keys(['cv', 'meta_regressor__copy_X', 'meta_regressor__fit_intercept', 'meta_regressor__n_jobs', 'meta_regressor__normalize', 'meta_regressor__positive', 'meta_regressor', 'multi_output', 'n_jobs', 'pre_dispatch', 'random_state', 'refit', 'regressors', 'shuffle', 'store_train_meta_features', 'use_features_in_secondary', 'verbose', 'randomforestregressor', 'gradientboostingregressor', 'linearregression', 'randomforestregressor__bootstrap', 'randomforestregressor__ccp_alpha', 'randomforestregressor__criterion', 'randomforestregressor__max_depth', 'randomforestregressor__max_features', 'randomforestregressor__max_leaf_nodes', 'randomforestregressor__max_samples', 'randomforestregressor__min_impurity_decrease', 'randomforestregressor__min_samples_leaf', 'randomforestregressor__min_samples_split', 'randomforestregressor__min_weight_fraction_leaf', 'randomforestregressor__n_estimators', 'randomforestregressor__n_jobs', 'randomforestregressor__oob_score', 'randomforestregressor__random_s

In [17]:
grid.best_score_, grid.best_params_

(0.9497603312176492, {'randomforestregressor__max_depth': 8})

In [21]:
# __init__() got an unexpected keyword argument 'randomforestregressor__max_depth'
# почему???
sr = StackingCVRegressor(regressors=[reg1, reg2, reg3], meta_regressor=meta_learner, randomforestregressor__max_depth=8)
sr.fit(X_train, y_train)
prediction = sr.predict(X_test)

TypeError: __init__() got an unexpected keyword argument 'randomforestregressor__max_depth'

In [ ]:
mean_squared_error(y_test, prediction)